In [24]:
# =========================
# Imports
# =========================

# --- Standard library 
from datetime import datetime
import re
import json


# --- Third-party ---
from IPython.display import Markdown, display
from openai import OpenAI

# --- Local / project ---
import tools.research_tools as research_tools

import os
from dotenv import find_dotenv, load_dotenv
# Load environment variables
load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
client = OpenAI()

In [154]:
def planner_agent(topic: str, model: str = "gpt-5", max_steps: int = 5) -> list[str]:
    """
    Generates a plan as a Python list of steps (strings) for a research workflow.

    Args:
        topic (str): Research topic to investigate.
        model (str): Language model to use.

    Returns:
        List[str]: A list of executable step strings.
    """
    prompt = f"""
You are a planning agent responsible for organizing a research workflow with multiple intelligent agents.

🧠 Available agents:
- A research agent who can search the web, Wikipedia, and arXiv.
- A writer agent who can draft research summaries.
- An editor agent who can reflect and revise the drafts.

🎯 Your job is to write a clear, step-by-step research plan **as a valid Python list**, where each step is a string.
Each step should be atomic, executable, and must rely only on the capabilities of the above agents.

🚫 DO NOT include irrelevant tasks like "create CSV", "set up a repo", "install packages", etc.
✅ DO include real research-related tasks (e.g., search, summarize, draft, revise).
✅ DO limit the search from few relevant sources.
✅ DO assume tool use is available.
🚫 DO NOT include explanation text — return ONLY the Python list.
✅ The final step should be to generate a Markdown document containing the complete and concise research report (title, introduction, findings, conclusion, references).
🚫 DO NOT ask any questions on the next steps at the end of the final Markdown document.

Topic: "{topic}"

Limit planning into {max_steps} steps.
"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
    )

    # ⚠️ Evaluate only if the environment is safe
    steps = eval(response.choices[0].message.content.strip())
    used_tokens = response.usage.total_tokens
    print("Used Tokens:\n", used_tokens)
    return steps


In [155]:
steps = planner_agent("AG Mill liner wear vs operational parameters")

Used Tokens:
 2495


In [156]:
steps

["Research agent: Search the web, Wikipedia, and arXiv for 'AG mill liner wear', 'autogenous mill liner wear', and 'AG/SAG liner wear operational parameters'; select the 5 most authoritative and directly relevant sources (e.g., SAG Conference papers, Minerals Engineering articles, Metso Outotec or FLSmidth technical notes), saving links and full citations.",
 'Research agent: From each selected source, extract key findings and any quantitative relationships linking liner wear to operational parameters (mill speed %Cs, mill filling/load, throughput, feed size/hardness, slurry density/chemistry, discharge/grate design), noting mill context, ranges, and page/figure references, and compile brief annotations.',
 'Writer agent: Synthesize the extracted evidence into a structured draft organized by parameter and wear mechanism (abrasion, impact, corrosion), compare and reconcile findings across sources, integrate quantitative values with in-text citations, and outline practical implications a

In [170]:
def writer_agent(task: str, model: str = "gpt-5-mini") -> str:
    """
    Executes writing tasks, such as drafting, expanding, or summarizing text.
    """
    print("==================================")
    print("✍️ Writer Agent")
    print("==================================")
    messages = [
        {"role": "system", "content": "You are a writing agent specialized in generating well-structured academic or technical content. DO NOT provde recommendations for the next steps at the end. DO NOT ask any questions on the next steps at the end."},
        {"role": "user", "content": task}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    used_tokens = response.usage.total_tokens
    print("Used Tokens:\n", used_tokens)
    return response.choices[0].message.content, used_tokens


In [171]:
def editor_agent(task: str, model: str = "gpt-5-mini") -> str:
    """
    Executes editorial tasks such as reflection, critique, or revision.
    """
    print("==================================")
    print("🧠 Editor Agent")
    print("==================================")
    messages = [
        {"role": "system", "content": "You are an editor agent. Your job is to reflect on, critique, or improve existing drafts. DO NOT provde recommendations for the next steps at the end. DO NOT ask any questions on the next steps at the end."},
        {"role": "user", "content": task}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    used_tokens = response.usage.total_tokens
    print("Used Tokens:\n", used_tokens)
    return response.choices[0].message.content, used_tokens


In [172]:
def research_agent(task: str, model: str = "gpt-5-mini", max_tool_call: int = 3):
    """
    Execute a research task using tools with aisuite (without manual loop).
    """
    print("==================================")
    print("🔍 Research Agent")
    print("==================================")
    tool_calls = 0
    prompt = f"""
You are a research assistant with 

Task:
{task}

Date today is {datetime.now().strftime('%Y-%m-%d')}
Limit tool calling into {max_tool_call} maximum.

"""
    def run_tool(name, args):
        if name == "arxiv_search_tool":
            return research_tools.arxiv_search_tool(**args)
        if name == "tavily_search_tool":
            return research_tools.tavily_search_tool(**args)
        if name == "wikipedia_search_tool":
            return research_tools.wikipedia_search_tool(**args)
        return {"error": f"Unknown tool: {name}"}

    messages = [
        {"role": "system", "content": """
            You are an expert researcher specialized in gathering academic or technical content. You have access to the following tools:
            - arxiv_tool: for finding academic papers
            - tavily_tool: for general web search
            - wikipedia_tool: for encyclopedic knowledge 
            ✅ Make the research concise.
            🚫 DO NOT provde recommendations for the next steps at the end. 
            🚫 DO NOT ask any questions on the next steps at the end.
            """},
        {"role": "user", "content":prompt.strip()}]
    tools = [research_tools.arxiv_tool_def, research_tools.tavily_tool_def, research_tools.wikipedia_tool_def]
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        msg = response.choices[0].message

        # messages.append(msg)
        kept = msg.tool_calls[:max_tool_call]
        sanitized_assistant = {
            "role": "assistant",
            "content": msg.content,
            "tool_calls": kept,
        }
        messages.append(sanitized_assistant)
        for call in msg.tool_calls:
            tool_calls += 1
            print(call.function.name, call.function.arguments)
            if tool_calls <= max_tool_call:
                result = run_tool(call.function.name, json.loads(call.function.arguments))
                messages.append({
                    "role": "tool",
                    "tool_call_id": call.id,
                    "name": call.function.name,
                    "content": json.dumps(result)
                })
        if tool_calls > 3:
            messages.append({
                "role": "assistant",
                "content": "I have reached the maximum tool usage as instructed. ",
            })
            
        final_response = client.chat.completions.create(
            model=model,
            messages=messages,
            reasoning_effort = "minimal"
        )
        
        content = final_response.choices[0].message.content
        print("✅ Output:\n", content)
        used_tokens = response.usage.total_tokens
        print("Used Tokens:\n", used_tokens)
        return content, used_tokens

    except Exception as e:
        print("❌ Error:", e)
        return f"[Model Error: {str(e)}]"


In [173]:


# prompt = """
# You are a research assistant with access to the following tools:
# - arxiv_tool: for finding academic papers
# - tavily_tool: for general web search
# - wikipedia_tool: for encyclopedic knowledge

# Task:
# You are research_agent.

#         Here is the context of what has been done so far:
        

#         Your next task is:
#         Search arXiv, Google Scholar, and Wikipedia for seminal papers and overviews on the ensemble Kalman filter for time series forecasting
    
# """
# research_agent(prompt)

In [174]:
agent_registry = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "writer_agent": writer_agent,
    # puedes agregar más si lo deseas
}

def clean_json_block(raw: str) -> str:
    """
    Clean the contents of a JSON block that may come wrapped with Markdown backticks.
    """
    raw = raw.strip()
    # Quitar bloque tipo ```json ... ```
    if raw.startswith("```"):
        raw = re.sub(r"^```(?:json)?\n?", "", raw)
        raw = re.sub(r"\n?```$", "", raw)
    return raw.strip()


In [175]:
def executor_agent(plan_steps: list[str], model: str = "gpt-5-nano"):
    history = []

    print("==================================")
    print("🎯 Execution Agent")
    print("==================================")
    total_used_token = 0
    
    for i, step in enumerate(plan_steps):
        # Paso 1: Determinar el agente y la tarea
        agent_decision_prompt = f"""
        You are an execution manager for a multi-agent research team.
        
        Given the following instruction, identify which agent should perform it and extract the clean task.
        
        Return only a valid JSON object with two keys:
        - "agent": one of ["research_agent", "editor_agent", "writer_agent"]
        - "task": a string with the instruction that the agent should follow
        
        Only respond with a valid JSON object. Do not include explanations or markdown formatting.
        
        Instruction: "{step}"
        """
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": agent_decision_prompt}]
            )
        
        # 🧼 Limpieza del bloque JSON
        raw_content = response.choices[0].message.content
        cleaned_json = clean_json_block(raw_content)
        agent_info = json.loads(cleaned_json)
        
        agent_name = agent_info["agent"]
        task = agent_info["task"]

        # Paso 2: Construir el contexto con outputs anteriores
        context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}" 
            for j, (s, a, r) in enumerate(history)
        ])
        enriched_task = f"""You are {agent_name}.
        
        Here is the context of what has been done so far:
        {context}
        
        Your next task is:
        {task}
        """

        print(f"\n🛠️ Executing with agent: `{agent_name}` on task: {task}")

        # Paso 3: Ejecutar el agente correspondiente
        if agent_name in agent_registry:
            output, used_token = agent_registry[agent_name](enriched_task)
            history.append((step, agent_name, output))
            total_used_token += used_token
            print(f"✅ Agent Used Token:\n{used_token}")
            
        else:
            output, used_token = f"⚠️ Unknown agent: {agent_name}"
            history.append((step, agent_name, output))
            total_used_token += used_token
            print(f"✅ Agent Used Token:\n{used_token}")
            
    print(f"✅ Output:\n{output}")
    print(f"✅ Total Used Token:\n{total_used_token}")
        
    return history, total_used_token



In [176]:
executor_history, total_used_token = executor_agent(steps)

🎯 Execution Agent

🛠️ Executing with agent: `research_agent` on task: Search the web, Wikipedia, and arXiv for 'AG mill liner wear', 'autogenous mill liner wear', and 'AG/SAG liner wear operational parameters'; select the 5 most authoritative and directly relevant sources (e.g., SAG Conference papers, Minerals Engineering articles, Metso Outotec or FLSmidth technical notes), saving links and full citations.
🔍 Research Agent
tavily_search_tool {"query": "AG mill liner wear autogenous mill liner wear SAG/AG liner wear operational parameters Metso Outotec FLSmidth technical note Minerals Engineering SAG Conference", "max_results": 10, "include_images": false}
arxiv_search_tool {"query": "autogenous mill liner wear OR AG mill liner wear OR SAG liner wear", "max_results": 5}
wikipedia_search_tool {"query": "Autogenous mill", "sentences": 5}
✅ Output:
 I searched the web, arXiv and Wikipedia for authoritative sources on AG/SAG/autogenous mill liner wear and operational parameters. Below are 

In [177]:
md = executor_history[-1][-1].strip("`")  
print(f"✅ Total Used Token:\n{total_used_token}")
display(Markdown(md))

✅ Total Used Token:
30053


# AG Mill Liner Wear vs Operational Parameters

## Introduction
This report synthesizes findings from five selected sources (DEM simulation study, an industry case paper, OEM technical literature and background material) on how operational parameters influence liner wear mechanisms in semi-autogenous grinding (SAG) mills. Primary wear mechanisms considered are: impact (fatigue) damage, abrasive wear, and corrosion / tribocorrosion. Emphasis is on directional effects and reported quantitative examples while noting that numerical results are generally site- and model-specific.

Sources used (full citations in References) include:
- Saxena et al., COMMINUTION ’23 DEM study with computed fatigue-life comparisons and a damage-rate index [1].
- An industry/engineering paper on liner-wear effects and plant case studies (ResearchGate) [2].
- OEM technical guidance and case materials from FLSmidth [3] and Metso Outotec [4].
- Background definitions and typical operating ranges from Wikipedia [5].

## Findings (organized by parameter and mechanism)

Overview
- Consensus across sources: mill speed, charge composition (total charge and ball/pebble fraction), liner geometry/state, feed size/hardness and discharge/grate condition are the primary operational controls of liner loading and wear. OEMs and academic work emphasize that quantitative prediction requires site-specific modelling or measurement.

### A. Mill speed (% of critical speed, Nc)
Mechanisms affected
- Impact / fatigue: Primary control. Higher speed generally increases particle/media cataracting, impact energy and impulse frequency → higher fatigue loading on liners and bolts.
- Abrasion: Indirect; speed changes shift charge motion and contact sliding regimes, altering abrasive exposure.
- Corrosion: Indirect via changed slurry hold-up patterns.

Quantitative notes and provenance
- OEM/background guidance: SAG mills commonly operate in the approximate 65–80% Nc band; many references cite ~70–80% Nc for SAG duties [3,4,5].
- Saxena et al. (DEM cases): modest rpm changes in their modeled mill (examples ≈ 9.0–9.6 rpm in the draft) produced large sensitivity in computed fatigue life. Reported fatigue-life estimates across modeled cases ≈ 244–533 days, with life varying up to ~2× for some speed/charge/profile combinations [1].

Operational implication (directional)
- Increasing speed → increased impact severity and reduced fatigue life (subject to interactions with charge and liner profile).

### B. Total charge and ball/pebble fraction (Jt, Jb)
Mechanisms affected
- Impact / fatigue: Strong control. Charge composition determines cushioning vs. discrete impacts and influences impact frequency and intensity.
- Abrasion: Filling/media fraction affects the balance of impact breakage vs attrition and thus sliding/abrasive exposure.

Quantitative notes and provenance
- Saxena et al.: DEM runs showed configurations with higher total charge and lower ball (pebble) fraction produced lower impact-damage indices and longer computed fatigue life in their modeled scenarios [1].
- OEMs identify charge makeup as a primary lever for controlling impact severity and liner loading [3,4].

Operational implication (directional)
- Higher total charge with lower added-ball fraction can reduce impact damage indices in some cases (DEM-backed) but outcomes are mill- and ore-specific.

### C. Liner geometry, profile and wear state
Mechanisms affected
- Impact / fatigue: Worn liners reduce lift and change impact locations; changing profile shifts cataracting vs rolling regimes and the distribution of high-energy impacts.
- Abrasion: Flattened/worn profiles increase regions of sliding/attrition and local abrasive thinning.

Quantitative notes and provenance
- Saxena et al.: Showed distinct damage-rate index distributions and different fatigue-life estimates for nominal, half-worn and fully-worn liner geometries in DEM comparisons [1].
- Industry case studies report throughput and grind changes as liners evolve (new → half-worn → fully worn) due to altered charge motion and impact/attrition balance [2].

Operational implication (directional)
- Liner wear leads to altered charge dynamics, tending to reduce cataracting and increase rolling/attrition, with consequences for both wear mode and production performance.

### D. Feed size and hardness
Mechanisms affected
- Abrasion: Primary control. Coarser and harder feed increases abrasive severity and accelerates liner thinning.
- Impact / fatigue: Coarse/hard feed benefits from cataracting impact; insufficient liner lift/profile to promote cataracting reduces breakage efficiency.

Quantitative notes and provenance
- Industry paper: Case-specific plots show throughput and power dependence on feed size in conjunction with liner-wear stage; no universal formula—results are plant dependent [2].
- OEM literature: Advises matching liner profile/material to feed characteristics to control wear and maintain grinding efficiency [3,4].

Operational implication (directional)
- Hard/coarse feed → higher abrasive wear; requires liner/profile/material choices that sustain desired charge motion.

### E. Discharge / grate condition and pulp retention
Mechanisms affected
- Abrasion & operational performance: Grate opening/wear alters pulp level and retention time → changes slurry exposure of liners and circulating load/classification behavior.

Quantitative notes and provenance
- Industry case studies: Increasing grate aperture or grate wear modifies internal pulp level and retention; throughput/power changes are case-specific (power changes often a few percent; throughput effects depend on feed and circuit) [2].

Operational implication (directional)
- Worn grates or altered discharge geometry change slurry distribution and can locally increase abrasive/tribocorrosive exposure.

### F. Liner material choice (steel, rubber, composite)
Mechanisms affected
- Abrasion: Different materials and surface finishes exhibit different sliding/abrasion resistance.
- Impact / fatigue: Material toughness and attachment systems influence fatigue resistance and failure modes.
- Corrosion: Non-metallic liners (rubber/composite) are less sensitive to chemical attack but have different wear trade-offs.

Quantitative notes and provenance
- OEM case studies report material/profile choices that yield measurable life improvements in specific installations; numbers are case-specific and presented as examples rather than universal claims [3,4].

Operational implication (directional)
- Material selection is a trade-off across abrasion resistance, impact toughness and chemical stability; site chemistry, feed and duty must inform choice.

### G. Corrosion and tribocorrosion (chemical environment)
Mechanisms affected
- Corrosion / tribocorrosion: Chemical attack (pH, oxidants, sulfides, chlorides) couples with mechanical action to accelerate metallic liner loss.

Quantitative notes and provenance
- OEMs treat pulp chemistry and residence time as inputs to liner material selection and monitoring; academic/industry reviewed sources do not provide general corrosion-rate equations for liners [3,4].

Operational implication (directional)
- Aggressive chemistry and longer slurry contact in localized pooling zones raise corrosion-assisted wear risk.

## Conclusion — Key takeaways and gaps

Key takeaways
- Directional effects are consistent and well-supported:
  - Mill speed is the dominant operational lever for impact/fatigue damage (higher speed → higher impact severity).
  - Charge composition (total charge and ball/pebble fraction) strongly modulates impact vs cushioning behavior.
  - Liner geometry and wear state have first-order effects on charge motion and therefore on both impact and abrasive wear regimes.
  - Feed size/hardness and discharge/grate condition substantially influence abrasive wear and slurry exposure patterns.
  - Liner material selection mitigates some wear modes but involves trade-offs among abrasion resistance, impact toughness and chemical stability.
- Quantitative examples exist but are case-specific:
  - Saxena et al.’s DEM study reports modeled fatigue-life estimates ≈ 244–533 days across parameter combinations; this demonstrates sensitivity but is tied to that model and mill geometry.
  - OEM guidance supplies operational bands (SAG mill speeds commonly in the ~65–80% Nc range; typical fill fractions often ~30–45% of mill volume) but emphasizes site-specific design and modelling.

Principal gaps and limitations in the available material
- Lack of universal predictive wear-rate equations: Quantitative wear rates and fatigue-life predictions depend on mill geometry, ore mechanics, liner material and modelling assumptions; available sources provide case-study or model-specific outputs rather than general formulas.
- Model and data limitations: DEM results depend on particle-size representation, contact laws and calibration; plant case studies are plant- and circuit-specific and frequently lack full public datasets.
- Incomplete published field datasets: Broad public datasets linking liner wear rates to systematically varied operational parameters across multiple plants are limited; OEM case studies and industry reports are illustrative but not comprehensive.

References
1. Saxena A., Vidaurre D., Molero O. (2023). Quantifying SAG mill liner survivability and wear life. COMMINUTION ’23 (conference draft). https://www.min-eng.com/comminution23/drafts/session5/saxena.pdf

2. "Understanding the effects of liner wear on SAG mill performance." Industry/engineering paper (ResearchGate). Verify authorship/publication details in the PDF. https://www.researchgate.net/publication/284600108_Understanding_the_effects_of_liner_wear_on_SAG_mill_performance

3. FLSmidth A/S. SAG and AG Mills — Product and technical information (liner selection, speed/fill guidance, maintenance). https://fls.com/en/equipment/grinding/sag-and-ag-mills

4. Metso Outotec Oyj. AG/SAG Mills and mill liners — technical brochures and service notes (liner materials, monitoring and case studies). https://www.metso.com/products/grinding/

5. Wikipedia contributors. "Mill (grinding)." Wikipedia, The Free Encyclopedia. Retrieved 2025-10-23. https://en.wikipedia.org/wiki/Mill_(grinding)